# Умная система фильтрации отзывов - Revi AI

Интеллектуальный микросервис для анализа отзывов, который  не только классифицирует обратную связь и определяет её тональность, но и автоматически выделяет ключевые проблемы и генерирует рекомендации для их решения

## Шаг 0. Загрузка данных и их обработка

## Шаг 1. Классификация спам-отзывов

Определяем, спам-отзыв это или нет. Также ставим порог (0.7) для сомнительных отзывов, чтобы те сохранялись в отдельный документ и переходили на дополнительную модерацию с аннотацией

In [ ]:
from transformers import pipeline

classifier = pipeline("text-classification", model="distilbert-base-uncased")

def classify_review(review):
    result = classifier(review)
    if result[0]['score'] < 0.7:
        return "Спорный"
    return result[0]['label']

## Шаг 2. Анализ тональности

Определяем тип отзыва - негативный / позитивный. Для негативных выставляем флаг `Спам, удалить`.

In [ ]:
from transformers import pipeline

sentiment_analyzer = pipeline("sentiment-analysis", model="roberta-base")

def analyze_sentiment(review):
    result = sentiment_analyzer(review)
    return result[0]['label']  # 'POSITIVE' or 'NEGATIVE'

## Шаг 3. Анализ контекста

Если отзыв имеет тип "Негативный", то выявляем проблемы. А если отзыв "Позитивный" - причины хорошего отзыва


### Выявление проблемы негативного отзыва

In [ ]:
problem_classifier = pipeline("text-classification", model="your-fine-tuned-model")

def identify_problem(review):
    result = problem_classifier(review)
    return result[0]['label']  # e.g., 'Долгая доставка'

### zero-shot подход

In [ ]:
from sentence_transformers import SentenceTransformer, util

# Шаг 1. Загрузка модели
model = SentenceTransformer('all-MiniLM-L6-v2')  # Легкая и быстрая модель

# Шаг 2. Список известных проблем
problem_list = [
    "Долгая доставка",
    "Холодная еда",
    "Не вежливый курьер",
    "Неправильный заказ",
    "Высокая цена",
]

# Создаем эмбеддинги проблем
problem_embeddings = model.encode(problem_list, convert_to_tensor=True)

# Шаг 3. Обработка отзыва
def match_problem(review_text):
    review_embedding = model.encode(review_text, convert_to_tensor=True)

    # Сравнение эмбеддингов
    similarities = util.cos_sim(review_embedding, problem_embeddings)

    # Находим наиболее похожую проблему
    max_sim_idx = similarities.argmax().item()
    max_similarity = similarities[0, max_sim_idx].item()

    # Устанавливаем порог сходства
    threshold = 0.5
    if max_similarity >= threshold:
        return problem_list[max_sim_idx], max_similarity
    else:
        return "Неопределенная проблема", max_similarity

# Пример
review = "Пиццу доставили через 2 часа, она была холодной"
problem, similarity = match_problem(review)
print(f"Определенная проблема: {problem} (Сходство: {similarity:.2f})")


### Выявление причины позитивного отзыва

In [ ]:
generator = pipeline("text2text-generation", model="t5-base")

def positive_reason(review):
    input_text = f"Why is this review positive? {review}"
    result = generator(input_text, max_length=50)
    return result[0]['generated_text']

## Шаг 4. Выделение эмоций

Оптимально использовать 6 основных эмоций: радость, грусть, гнев, страх, удивление, отвращение. Эти эмоции универсальны и покрывают большинство сценариев.

In [ ]:
from transformers import pipeline

emotion_analyzer = pipeline("text-classification", model="bhadresh-savani/distilbert-base-uncased-emotion")

def detect_emotion(review):
    emotions = emotion_analyzer(review)
    return max(emotions, key=lambda x: x['score'])['label']  # e.g., 'anger'

## Шаг 5. Генерация рекомендаций

Используем GPT для построения рекомендаций. Рекомендации можно отправлять ресторанам - партнерам для лучшей коммуникации.


In [ ]:
def generate_recommendation(problem):
    prompt = f"Provide a recommendation for the problem: {problem}"
    result = generator(prompt, max_length=50)
    return result[0]['generated_text']